# Criteo Click Through Prediction using ThirdAI's Universal Deep Transformer(UDT) APIs
This notebook shows how to build a Click Through Prediction model using ThirdAI's UDT.

In [1]:
!pip3 install thirdai --upgrade

Requirement already up-to-date: thirdai in /home/pratik/.local/lib/python3.8/site-packages (0.5.10)


# Download and Process Dataset

We will now create a sample dataset for training. Criteo-Click Logs dataset can be downloaded from https://ailab.criteo.com. Once datasets are downloaded convert the datasets into csv format. 

Column headers: `<label>` is titled as `<label>`, `<integer feature 1>` is titled as `numeric_1` and `<categorical feature 1>` as `cat_1`

<span style="color:red">Disclaimer: Remember that the following model is trained on sample data, to achieve better AUC train on the original criteo-dataset.</span>

In [2]:
import numpy as np
import pandas as pd

TRAIN_FILE = 'train_file.txt'
TEST_FILE = 'test_file.txt'

header = 'label,numeric_1,numeric_2,numeric_3,numeric_4,numeric_5,numeric_6,numeric_7,numeric_8,numeric_9,numeric_10,numeric_11,numeric_12,numeric_13,'
header += 'cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,'
header += 'cat_22,cat_23,cat_24,cat_25,cat_26'

column_header=header.split(",")

num_training_data=1200
num_numeric_features=13
num_categorical_features=26

numeric_data = np.random.randint(1500, size=(num_training_data, num_numeric_features))
categorical_data = np.random.randint(10000, size=(num_training_data, num_categorical_features))
label = np.random.randint(2, size=(num_training_data, 1))

data_X , data_Y = np.concatenate((numeric_data, categorical_data), axis=1), label

train_data = np.concatenate((data_Y[:1000], data_X[:1000]), axis=1)
train_df = pd.DataFrame(train_data, columns=column_header)
train_df.to_csv(TRAIN_FILE, index=False)


test_data = np.concatenate((data_Y[1000:], data_X[1000:]), axis=1)
train_df = pd.DataFrame(test_data, columns=column_header)
train_df.to_csv(TEST_FILE, index=False)

train_df.head(10)

,label,numeric_1,numeric_2,numeric_3,numeric_4,numeric_5,numeric_6,numeric_7,numeric_8,numeric_9,...,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23,cat_24,cat_25,cat_26
0,1,172,1449,419,380,790,1428,554,1416,180,...,4732,1180,1674,4219,3116,5981,1321,1727,7354,6595
1,0,118,602,788,1028,937,1014,1233,1464,20,...,7273,4066,2962,9159,4573,6973,2731,505,2609,7230
2,0,286,1390,1120,114,1117,329,223,1270,1096,...,2148,1668,7751,3172,1895,8686,9627,8358,6268,6087
3,1,1344,900,573,165,615,1378,925,183,131,...,3383,8284,1908,1196,3414,3159,4526,8768,8279,8759
4,0,938,1008,956,529,939,641,1180,1112,956,...,7415,8323,9040,9575,7245,6061,3094,2919,439,9524
5,0,635,682,675,875,1133,550,304,352,1268,...,4881,2620,7528,5253,3969,2207,7800,9556,6958,1048
6,0,1200,545,176,1465,352,651,273,1214,684,...,6064,253,2896,553,1261,2652,3202,3430,8989,1631
7,1,1448,872,862,615,363,1461,276,1144,1042,...,1008,8670,2815,4894,2699,7541,3819,4719,2240,4679
8,1,899,838,345,176,930,1498,396,460,1231,...,1926,2472,5936,9453,6103,1188,6190,7655,421,6305
9,0,1498,571,1454,769,1209,434,75,1343,454,...,117,6942,641,4351,4801,7631,5895,8400,6289,5232


# UDT Initialization


We can create a UDT model specific for Criteo Click Through Prediction as follows. Here we define Bolt Datatype for all the columns, specify the target, n_target classes and the embedding dimension for UDT. 

In [3]:
from thirdai import bolt
import numpy as np
from sklearn.metrics import roc_auc_score


tabular_model = bolt.UniversalDeepTransformer(
    data_types={
        "numeric_1": bolt.types.numerical(range=(0, 1500)),
        "numeric_2": bolt.types.numerical(range=(0, 1500)),
        "numeric_3": bolt.types.numerical(range=(0, 1500)),
        "numeric_4": bolt.types.numerical(range=(0, 1500)),
        "numeric_5": bolt.types.numerical(range=(0, 1500)),
        "numeric_6": bolt.types.numerical(range=(0, 1500)),
        "numeric_7": bolt.types.numerical(range=(0, 1500)),
        "numeric_8": bolt.types.numerical(range=(0, 1500)),
        "numeric_9": bolt.types.numerical(range=(0, 1500)),
        "numeric_10": bolt.types.numerical(range=(0, 1500)),
        "numeric_11": bolt.types.numerical(range=(0, 1500)),
        "numeric_12": bolt.types.numerical(range=(0, 1500)),
        "numeric_13": bolt.types.numerical(range=(0, 1500)),
        "cat_1": bolt.types.categorical(),
        "cat_2": bolt.types.categorical(),
        "cat_3": bolt.types.categorical(),
        "cat_4": bolt.types.categorical(),
        "cat_5": bolt.types.categorical(),
        "cat_6": bolt.types.categorical(),
        "cat_7": bolt.types.categorical(),
        "cat_8": bolt.types.categorical(),
        "cat_9": bolt.types.categorical(),
        "cat_10": bolt.types.categorical(),
        "cat_11": bolt.types.categorical(),
        "cat_12": bolt.types.categorical(),
        "cat_13": bolt.types.categorical(),
        "cat_14": bolt.types.categorical(),
        "cat_15": bolt.types.categorical(),
        "cat_16": bolt.types.categorical(),
        "cat_17": bolt.types.categorical(),
        "cat_18": bolt.types.categorical(),
        "cat_19": bolt.types.categorical(),
        "cat_20": bolt.types.categorical(),
        "cat_21": bolt.types.categorical(),
        "cat_22": bolt.types.categorical(),
        "cat_23": bolt.types.categorical(),
        "cat_24": bolt.types.categorical(),
        "cat_25": bolt.types.categorical(),
        "cat_26": bolt.types.categorical(),
        "label": bolt.types.categorical(),
    },
    target="label",
    n_target_classes=2,
    options={"embedding_dimension": 512},
)

# Training

We will now train the UDT with just one line of code. Here we are specifying the training file name, number of epochs to train and maximum number of batches we want to load in memory at once. `max_in_memory_batches` support loaing data in a streaming fashion. You can change `max_in_memory_batches` based on your own memory specification.

In [4]:


tabular_model.train(filename=TRAIN_FILE, epochs=1, max_in_memory_batches=1000)



loading data | source 'train_file.txt' | vectors 1000 | batches 1 | time 0s | complete

train | epoch 0 | train_steps 1 | {} | train_batches 1 | time 0s | complete     



# Evaluation

Evaluating the UDT is also just one line of code. Evaluate function by default returns activations. Then, we are using sklearn's roc_auc_score to calculate roc_auc_score for the model we trained.

In [5]:
activations = tabular_model.evaluate(filename=TEST_FILE, metrics=["categorical_accuracy"])

true_labels = np.zeros(activations.shape[0], dtype=np.float32)
with open(TEST_FILE) as f:
    header = f.readline()
    count = 0
    for line in f:
        true_labels[count] = np.float32(line.split(",")[0])
        count += 1

roc_auc = roc_auc_score(true_labels, activations[:, 1])

print("ROC_AUC:", roc_auc)



loading data | source 'test_file.txt' | vectors 200 | batches 1 | time 0s | complete

evaluate | epoch 1 | train_steps 1 | {categorical_accuracy: 0.43} | eval_batches 1 | time 8ms

ROC_AUC: 0.4782741738066095


In [6]:
import os
os.remove(TRAIN_FILE)
os.remove(TEST_FILE)